In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_LiH_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23594/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘           │        »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«     ┌┴────────────┴┐   ┌────────────┐                  
«q_1: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘  ┌┴────────────┴┐ ┌──────────────┐
«q_2: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_3: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«                     └─────────────────┘ └────────────┘ 
«q_4: ───────────────────────────────────────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [0.008239051732521298, 0.013635804359443573, 0.05162548888524782, -2.1999413989625588e-05]
phase_z_front = [-0.010143905866716232, -0.028142425222093263, 0.008659821134257232, 1.4378429162081177]
phase_y_end = [0.008261085257040952, 0.013555948647231175, 0.0516520269898158, -4.555585169085397e-05]
phase_z_end = [1.3183908530236275, 1.3352666291100785, -0.26460520551963923, 1.2198754282688027]
x_front = [True, True, False, False]

In [18]:
qubit_id = 3
circuit = QuantumCircuit(5)
apply_checking_circuit(circuit, [qubit_id], [4], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [4], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23594/2509244131.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────────────»
«     └───────────────┘   ┌┴────────────┴┐ ┌────────────┐                      »
«q_1: ────────■───────────┤ Ry(0.013556) ├─┤ Rz(1.3353) ├──────────────────────»
«             │           └──────────────┘ └────────────┘     ┌──────────────┐ »
«q_2: ────────■───────────────────────────────────────────■───┤ Ry(0.051652) ├─»
«     ┌────────────────┐┌─────────────────┐┌────────────┐ │ ┌─┴──────────────┴┐»
«q_3: ┤ Ry(2.1999e-05) ├┤ Ry(-2.1999e-05) ├┤ Rz(1.4378) ├─■─┤ Ry(-4.5556e-05) ├»
«     └────────────────┘└─────────────────┘└────────────┘   └─────────────────┘»
«q_4: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_1: ───────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐                                                       »
«q_2: ┤ Rz(-0.26461) ├───────────────────────────────────────────────────────»
«     └┬────────────┬┘┌─────────────┐┌────────────────┐   ┌─────────────────┐»
«q_3: ─┤ Rz(1.2199) ├─┤ Rz(-1.2199) ├┤ Ry(4.5556e-05) ├─■─┤ Ry(-4.5556e-05) ├»
«      └────────────┘ └─────────────┘└────────────────┘ │ └──────┬───┬──────┘»
«q_4: ──────────────────────────────────────────────────■────────┤ H ├───────»
«                                                                └───┘       »
«                   
«q_0: ──────────────
«                   
«q_1: ──────────────
«                   
«q_2: ──────────────
«     ┌────────────┐
«q_3: ┤ Rz(1.2199) ├
«     └────────────┘
«q_4: ──────────────
«

In [20]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
q_4: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXX
     ┌─

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23594/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────────────»
«     └───────────────┘   ┌┴────────────┴┐ ┌────────────┐                      »
«q_1: ────────■───────────┤ Ry(0.013556) ├─┤ Rz(1.3353) ├──────────────────────»
«             │           └──────────────┘ └────────────┘     ┌──────────────┐ »
«q_2: ────────■───────────────────────────────────────────■───┤ Ry(0.051652) ├─»
«     ┌────────────────┐┌─────────────────┐┌────────────┐ │ ┌─┴──────────────┴┐»
«q_3: ┤ Ry(2.1999e-05) ├┤ Ry(-2.1999e-05) ├┤ Rz(1.4378) ├─■─┤ Ry(-4.5556e-05) ├»
«     └────────────────┘└─────────────────┘└────────────┘   └─────────────────┘»
«q_4: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_1: ───────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐                                                       »
«q_2: ┤ Rz(-0.26461) ├───────────────────────────────────────────────────────»
«     └┬────────────┬┘┌─────────────┐┌────────────────┐   ┌─────────────────┐»
«q_3: ─┤ Rz(1.2199) ├─┤ Rz(-1.2199) ├┤ Ry(4.5556e-05) ├─■─┤ Ry(-4.5556e-05) ├»
«      └────────────┘ └─────────────┘└────────────────┘ │ └──────┬───┬──────┘»
«q_4: ──────────────────────────────────────────────────■────────┤ H ├───────»
«                                                                └───┘       »
«                                 
«q_0: ────────────────────────────
«                                 
«q_1: ────────────────────────────
«                                 
«q_2: ────────────────────────────
«     ┌────────────┐┌────────────┐
«q_3: ┤ Rz(1.2199) ├┤ U(π/2,0,π) ├
«     └────────────┘└────────────┘
«q_4: ────────────────────────────
«

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[3], 5),(circuit.qubits[3], 10)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐  ┌──────────────┐                »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──┤ U(π/2,0,π/2) ├────────────────»
«     └──────────────

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐  ┌──────────────┐                »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──┤ U(π/2,0,π/2) ├────────────────»
«     └──────────────

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────────────»
«     └──────

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐  ┌────────────┐                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──┤ U(π/2,0,π) ├──────────────────────»
«     └──────

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐  ┌────────────┐              »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──┤ U(π/2,0,π) ├──────────────»
«     └───────────────┘   ┌┴─

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                              »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────»
«     └───────────────┘   ┌┴─

            ┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────────────»
«     └──────

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(3, 'q'), 2)), (0, Qubit(QuantumRegister(4, 'q'), 1)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌───┐        ┌──────────────┐┌───────────────┐                    »
q_0: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
     ┌──────┴───┴──────┐ └┬────────────┬┘└───────────────┘ │                  »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼──────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐ │ ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘   └───────────────┘»
q_3: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
       └──────────────┘ └───────────────┘                                     »
«        ┌──────────────┐┌────────────┐  ┌──────────────┐                 »
«q_0: ─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├──┤ U(π/2,0,π/2) ├─────────────────»
«      │ └──────────────┘└────────────┘┌─┴──────────────┴┐ ┌────────────┐ »
«q_1: ─┼───────────────────────■───────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─»
«      │  ┌────────────┐       │       └─┬──────────────┬┘ └────────────┘ »
«q_2: ─┼──┤ Rz(1.3184) ├───────┼─────────┤ U(π/2,0,π/2) ├─────────────────»
«      │  └────────────┘       │         ├──────────────┤ ┌──────────────┐»
«q_3: ─■───────────────────────■─────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├»
«                                        └──────────────┘ └──────────────┘»
«                     
«q_0: ────────────────
«                     
«q_1: ────────────────
«                     
«q_2: ────────────────
«     ┌──────────────┐
«q_3: ┤ U(π/2,0,π/2) ├
«     └──────────────┘

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
            ┌───┐        ┌──────────────┐┌───────────────┐                    »
q_0: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
     ┌──────┴───┴──────┐ └┬────────────┬┘└───────────────┘ │                  »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼──────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐ │ ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘   └───────────────┘»
q_3: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
       └──────────────┘ └───────────────┘                                     »
«        ┌──────────────┐┌────────────┐  ┌──────────────┐                 »
«q_0: ─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├──┤ U(π/2,0,π/2) ├─────────────────»
«      │ └──────────────┘└────────────┘┌─┴──────────────┴┐ ┌────────────┐ »
«q_1: ─┼───────────────────────■───────┤

            ┌───┐        ┌──────────────┐┌───────────────┐                    »
q_0: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
     ┌──────┴───┴──────┐ └┬────────────┬┘└───────────────┘ │                  »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼──────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐ │ ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘   └───────────────┘»
q_3: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
       └──────────────┘ └───────────────┘                                     »
«        ┌──────────────┐┌────────────┐                                   »
«q_0: ─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├───────────────────────────────────»
«      │ └──────────────┘└────────────┘┌─────────────────┐ ┌────────────┐ »
«q_1: ─┼───────────────────────■───────┤ Ry(-4.5556e

            ┌───┐        ┌──────────────┐┌───────────────┐                    »
q_0: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
     ┌──────┴───┴──────┐ └┬────────────┬┘└───────────────┘ │                  »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼──────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐ │ ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘   └───────────────┘»
q_3: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
       └──────────────┘ └───────────────┘                                     »
«        ┌──────────────┐┌────────────┐   ┌────────────┐                  »
«q_0: ─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├───┤ U(π/2,0,π) ├──────────────────»
«      │ └──────────────┘└────────────┘┌──┴────────────┴─┐ ┌────────────┐ »
«q_1: ─┼───────────────────────■───────┤ Ry(-4.5556e

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(3, 'q'), 2)),
  (0, Qubit(QuantumRegister(4, 'q'), 1)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [37]:
fragments[0].draw()

┌───┐        ┌──────────────┐┌───────────────┐                    »
q_0: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
     ┌──────┴───┴──────┐ └┬────────────┬┘└───────────────┘ │                  »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼──────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐ │ ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘   └───────────────┘»
q_3: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
       └──────────────┘ └───────────────┘                                     »
«        ┌──────────────┐┌────────────┐                                   
«q_0: ─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├───────────────────────────────────
«      │ └──────────────┘└────────────┘┌─────────────────┐ ┌────────────┐ 
«q_1: ─┼───────────────────────■───────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«      │  ┌────────────┐       │       └─────────────────┘ └────────────┘ 
«q_2: ─┼──┤ Rz(1.3184) ├───────┼──────────────────────────────────────────
«      │  └────────────┘       │         ┌──────────────┐ ┌──────────────┐
«q_3: ─■───────────────────────■─────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«                                        └──────────────┘ └──────────────┘

In [38]:
fragments[1].draw()

┌───┐                                                  ┌───┐       »
q_0: ───────┤ H ├──────────────────────────■────────■──────────────┤ H ├───────»
       ┌────┴───┴────┐  ┌────────────────┐ │        │       ┌──────┴───┴──────┐»
q_1: ──┤ Rz(-1.2199) ├──┤ Ry(4.5556e-05) ├─┼────────■───────┤ Ry(-4.5556e-05) ├»
     ┌─┴─────────────┴─┐└─┬────────────┬─┘ │ ┌─────────────┐└┬────────────────┤»
q_2: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───■─┤ Rz(-1.4378) ├─┤ Ry(2.1999e-05) ├»
     └─────────────────┘  └────────────┘     └─────────────┘ └────────────────┘»
«                   
«q_0: ──────────────
«     ┌────────────┐
«q_1: ┤ Rz(1.2199) ├
«     └────────────┘
«q_2: ──────────────
«

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

25

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
#16,14,11,8
#[14,8,16,11]

In [48]:
multi_exp_circuits[hardware_index][-200].draw()

░        ┌───┐        ┌──────────────┐»
  q_0: ────────────────────░────────┤ X ├────────┤ Ry(0.013636) ├»
       ┌─────────────────┐ ░ ┌──────┴───┴──────┐ └┬────────────┬┘»
  q_1: ┤ U3(-1.9106,π,0) ├─░─┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─»
       └─────────────────┘ ░ └──────┬───┬──────┘┌─┴────────────┴┐»
  q_2: ────────────────────░────────┤ X ├───────┤ Ry(0.0082391) ├»
                           ░   ┌────┴───┴─────┐ ├───────────────┤»
  q_3: ────────────────────░───┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├»
                           ░   └──────────────┘ └───────────────┘»
c16_0: ══════════════════════════════════════════════════════════»
                                                                 »
c16_1: ══════════════════════════════════════════════════════════»
                                                                 »
c16_2: ══════════════════════════════════════════════════════════»
                                                                 »
c16_3: ══════════════════════════════════════════════════════════»
                                                                 »
«       ┌───────────────┐                       ┌──────────────┐┌────────────┐»
«  q_0: ┤ Rz(-0.028142) ├─■───────────────────■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├»
«       └───────────────┘ │                   │ └──────────────┘└────────────┘»
«  q_1: ──────────────────┼───────────────────┼───────────────────────■───────»
«       ┌───────────────┐ │ ┌───────────────┐ │  ┌────────────┐       │       »
«  q_2: ┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├─┼──┤ Rz(1.3184) ├───────┼───────»
«       └───────────────┘   └───────────────┘ │  └────────────┘       │       »
«  q_3: ──────────────────────────────────────■───────────────────────■───────»
«                                                                             »
«c16_0: ══════════════════════════════════════════════════════════════════════»
«                                                                             »
«c16_1: ══════════════════════════════════════════════════════════════════════»
«                                                                             »
«c16_2: ══════════════════════════════════════════════════════════════════════»
«                                                                             »
«c16_3: ══════════════════════════════════════════════════════════════════════»
«                                                                             »
«                                                           ░ ┌─┐           
«  q_0: ────────────────────────────────────────────────────░─┤M├───────────
«       ┌─────────────────┐ ┌────────────┐                  ░ └╥┘┌───┐   ┌─┐
«  q_1: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├──────────────────░──╫─┤ H ├───┤M├
«       └─────────────────┘ └────────────┘                  ░  ║ └┬─┬┘   └╥┘
«  q_2: ────────────────────────────────────────────────────░──╫──┤M├─────╫─
«         ┌──────────────┐ ┌──────────────┐┌──────────────┐ ░  ║  └╥┘ ┌─┐ ║ 
«  q_3: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ U(π/2,0,π/2) ├─░──╫───╫──┤M├─╫─
«         └──────────────┘ └──────────────┘└──────────────┘ ░  ║   ║  └╥┘ ║ 
«c16_0: ═══════════════════════════════════════════════════════╩═══╬═══╬══╬═
«                                                                  ║   ║  ║ 
«c16_1: ═══════════════════════════════════════════════════════════╬═══╬══╩═
«                                                                  ║   ║    
«c16_2: ═══════════════════════════════════════════════════════════╩═══╬════
«                                                                      ║    
«c16_3: ═══════════════════════════════════════════════════════════════╩════
«

In [79]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [14,8,16,11],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  63918b1f004b54efd653384d
Job Status: job has successfully run
job_id:  733986eb-c3ca-4377-b892-1a88e27fc4a7
Job Status: job has successfully run


In [80]:
frag_datas[0]

[Result(backend_name='ibm_auckland', backend_version='1.3.0', qobj_id='9ff86118-dba5-49e1-8f4e-e9dd93d926ae', job_id='63918b1f004b54efd653384d', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 1634, '0x1': 1597, '0x2': 26, '0x3': 28, '0x4': 890, '0x5': 974, '0x6': 17, '0x7': 15, '0x8': 1388, '0x9': 1498, '0xa': 28, '0xb': 27, '0xc': 922, '0xd': 922, '0xe': 18, '0xf': 16}), header=QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1], ['c0', 2], ['c0', 3]], creg_sizes=[['c0', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=27, name="(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))", qreg_sizes=[['q', 27]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6], ['q', 7], ['q', 8], ['q', 9], ['q', 10], ['q', 11], ['q', 12], ['q', 13], ['q', 14], ['q', 15], ['q', 16], ['q', 17], ['q', 18], ['q', 19], ['q', 20], ['q', 21], ['q', 22], ['q', 23], ['q', 24], ['q', 25], ['q', 26

In [81]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [82]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [53]:
# managed_results = job_set.results()

In [104]:
frag_datas1_copy = frag_datas[1].copy()

In [105]:
frag_datas[1] = [frag_datas[1][0].combine_results()]

In [106]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [83]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [84]:
num_tomo_circuits

25

In [85]:
test = [[], []]

In [86]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [87]:
frag_datas[1][0].get_counts()

[{'000': 5001, '010': 4999},
 {'010': 2429, '110': 2578, '000': 2507, '100': 2486},
 {'010': 2513, '110': 2526, '000': 2477, '100': 2484},
 {'000': 1708, '011': 3215, '010': 1736, '001': 3341},
 {'111': 1660,
  '011': 1600,
  '101': 1630,
  '010': 810,
  '000': 880,
  '110': 854,
  '100': 853,
  '001': 1713},
 {'101': 1640,
  '001': 1696,
  '110': 846,
  '100': 815,
  '011': 1610,
  '111': 1691,
  '010': 844,
  '000': 858},
 {'001': 3394, '000': 1680, '010': 1624, '011': 3302},
 {'100': 825,
  '110': 843,
  '001': 1683,
  '000': 823,
  '010': 874,
  '101': 1652,
  '111': 1648,
  '011': 1652},
 {'101': 1646,
  '011': 1638,
  '111': 1706,
  '001': 1721,
  '100': 857,
  '110': 810,
  '010': 783,
  '000': 839},
 {'001': 3318, '010': 1697, '011': 3354, '000': 1631},
 {'101': 1697,
  '010': 865,
  '000': 848,
  '110': 806,
  '100': 824,
  '001': 1684,
  '111': 1676,
  '011': 1600},
 {'010': 836,
  '000': 827,
  '001': 1691,
  '100': 860,
  '110': 868,
  '101': 1668,
  '111': 1607,
  '011': 1

In [88]:
frag_datas[0][0].get_counts()

[{'0000': 1634,
  '0001': 1597,
  '0010': 26,
  '0011': 28,
  '0100': 890,
  '0101': 974,
  '0110': 17,
  '0111': 15,
  '1000': 1388,
  '1001': 1498,
  '1010': 28,
  '1011': 27,
  '1100': 922,
  '1101': 922,
  '1110': 18,
  '1111': 16},
 {'0000': 806,
  '0001': 783,
  '0010': 781,
  '0011': 770,
  '0100': 480,
  '0101': 532,
  '0110': 448,
  '0111': 508,
  '1000': 750,
  '1001': 766,
  '1010': 774,
  '1011': 741,
  '1100': 489,
  '1101': 480,
  '1110': 458,
  '1111': 434},
 {'0000': 758,
  '0001': 771,
  '0010': 801,
  '0011': 849,
  '0100': 462,
  '0101': 475,
  '0110': 480,
  '0111': 533,
  '1000': 737,
  '1001': 742,
  '1010': 735,
  '1011': 794,
  '1100': 405,
  '1101': 464,
  '1110': 469,
  '1111': 525},
 {'0000': 531,
  '0001': 550,
  '0010': 976,
  '0011': 984,
  '0100': 322,
  '0101': 318,
  '0110': 589,
  '0111': 614,
  '1000': 497,
  '1001': 498,
  '1010': 1167,
  '1011': 1009,
  '1100': 299,
  '1101': 327,
  '1110': 647,
  '1111': 672},
 {'0000': 167,
  '0001': 153,
  '0010'

In [89]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [1] [1]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'X', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0',) ('X',) [1] [1]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Y', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0',) ('Y',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'X', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S1',) ('X',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'Y', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S1',) ('Y',) [1] [1

In [90]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1


In [91]:
len(organized_data)

25

In [92]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 1393,
   (('S0',), ('Zm',)): 29,
   (('S0',), ('Xp',)): 708,
   (('S0',), ('Xm',)): 753,
   (('S0',), ('Yp',)): 723,
   (('S0',), ('Ym',)): 679,
   (('S1',), ('Zp',)): 455,
   (('S1',), ('Zm',)): 1198,
   (('S1',), ('Xp',)): 214,
   (('S1',), ('Xm',)): 1406,
   (('S1',), ('Yp',)): 1070,
   (('S1',), ('Ym',)): 522,
   (('S2',), ('Zp',)): 468,
   (('S2',), ('Zm',)): 1187,
   (('S2',), ('Xp',)): 1386,
   (('S2',), ('Xm',)): 226,
   (('S2',), ('Yp',)): 1240,
   (('S2',), ('Ym',)): 414,
   (('S3',), ('Zp',)): 498,
   (('S3',), ('Zm',)): 1209,
   (('S3',), ('Xp',)): 873,
   (('S3',), ('Xm',)): 749,
   (('S3',), ('Yp',)): 93,
   (('S3',), ('Ym',)): 1573},
  '001': {(('S0',), ('Zp',)): 1461,
   (('S0',), ('Zm',)): 37,
   (('S0',), ('Xp',)): 770,
   (('S0',), ('Xm',)): 746,
   (('S0',), ('Yp',)): 754,
   (('S0',), ('Ym',)): 787,
   (('S1',), ('Zp',)): 484,
   (('S1',), ('Zm',)): 1250,
   (('S1',), ('Xp',)): 186,
   (('S1',), ('Xm',)): 1538,
   (('S1',), ('Yp',)): 1

In [93]:
organized_data[-1][-1]

{'00': {(('S0',), ('Zp',)): 10000,
  (('S0',), ('Xp',)): 4930,
  (('S0',), ('Xm',)): 5070,
  (('S0',), ('Ym',)): 4977,
  (('S0',), ('Yp',)): 5023,
  (('S1',), ('Zp',)): 3234,
  (('S1',), ('Xm',)): 1680,
  (('S1',), ('Xp',)): 1725,
  (('S1',), ('Ym',)): 1635,
  (('S1',), ('Yp',)): 1615,
  (('S2',), ('Zp',)): 3411,
  (('S2',), ('Xm',)): 1649,
  (('S2',), ('Xp',)): 1706,
  (('S2',), ('Yp',)): 1670,
  (('S2',), ('Ym',)): 1681,
  (('S3',), ('Zp',)): 3334,
  (('S3',), ('Xm',)): 1670,
  (('S3',), ('Xp',)): 1675,
  (('S3',), ('Ym',)): 1671,
  (('S3',), ('Yp',)): 1657,
  (('S0',), ('Zm',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0,
  (('S3',), ('Zm',)): 0},
 '11': {(('S1',), ('Zp',)): 6766,
  (('S1',), ('Xm',)): 3218,
  (('S1',), ('Xp',)): 3377,
  (('S1',), ('Yp',)): 3341,
  (('S1',), ('Ym',)): 3409,
  (('S2',), ('Zp',)): 6589,
  (('S2',), ('Xm',)): 3296,
  (('S2',), ('Xp',)): 3349,
  (('S2',), ('Ym',)): 3383,
  (('S2',), ('Yp',)): 3266,
  (('S3',), ('Zp',)): 6666,
  (('S3',), ('Xm',

In [94]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 1634, (('S0',), ('Zm',)): 26, (('S0',), ('Xp',)): 806, (('S0',), ('Xm',)): 781, (('S0',), ('Yp',)): 758, (('S0',), ('Ym',)): 801, (('S1',), ('Zp',)): 531, (('S1',), ('Zm',)): 976, (('S1',), ('Xp',)): 167, (('S1',), ('Xm',)): 1376, (('S1',), ('Yp',)): 1015, (('S1',), ('Ym',)): 436, (('S2',), ('Zp',)): 553, (('S2',), ('Zm',)): 964, (('S2',), ('Xp',)): 1374, (('S2',), ('Xm',)): 190, (('S2',), ('Yp',)): 1126, (('S2',), ('Ym',)): 383, (('S3',), ('Zp',)): 547, (('S3',), ('Zm',)): 902, (('S3',), ('Xp',)): 816, (('S3',), ('Xm',)): 693, (('S3',), ('Yp',)): 74, (('S3',), ('Ym',)): 1476}, '001': {(('S0',), ('Zp',)): 1597, (('S0',), ('Zm',)): 28, (('S0',), ('Xp',)): 783, (('S0',), ('Xm',)): 770, (('S0',), ('Yp',)): 771, (('S0',), ('Ym',)): 849, (('S1',), ('Zp',)): 550, (('S1',), ('Zm',)): 984, (('S1',), ('Xp',)): 153, (('S1',), ('Xm',)): 1286, (('S1',), ('Yp',)): 1075, (('S1',), ('Ym',)): 469, (('S2',), ('Zp',)): 570, (('S2',), ('Zm',)): 972, (('S2',), ('Xp',)): 1350, 

{'00': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xm',)): 5022, (('S0',), ('Xp',)): 4978, (('S0',), ('Yp',)): 4969, (('S0',), ('Ym',)): 5031, (('S1',), ('Zp',)): 3352, (('S1',), ('Xm',)): 1681, (('S1',), ('Xp',)): 1676, (('S1',), ('Ym',)): 1681, (('S1',), ('Yp',)): 1702, (('S2',), ('Zp',)): 3356, (('S2',), ('Xm',)): 1631, (('S2',), ('Xp',)): 1666, (('S2',), ('Ym',)): 1639, (('S2',), ('Yp',)): 1702, (('S3',), ('Zp',)): 3341, (('S3',), ('Xm',)): 1559, (('S3',), ('Xp',)): 1603, (('S3',), ('Yp',)): 1646, (('S3',), ('Ym',)): 1649, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '11': {(('S1',), ('Zp',)): 6648, (('S1',), ('Xm',)): 3307, (('S1',), ('Xp',)): 3336, (('S1',), ('Ym',)): 3355, (('S1',), ('Yp',)): 3262, (('S2',), ('Zp',)): 6644, (('S2',), ('Xp',)): 3360, (('S2',), ('Xm',)): 3343, (('S2',), ('Ym',)): 3333, (('S2',), ('Yp',)): 3326, (('S3',), ('Zp',)): 6659, (('S3',), ('Xp',)): 3429, (('S3',), ('Xm',)): 3409, (('S3',), ('Yp',)): 3243, (('S3',

{'00': {(('S0',), ('Zp',)): 4971, (('S0',), ('Xm',)): 2485, (('S0',), ('Xp',)): 2459, (('S0',), ('Ym',)): 2471, (('S0',), ('Yp',)): 2467, (('S1',), ('Zp',)): 1681, (('S1',), ('Xm',)): 832, (('S1',), ('Xp',)): 826, (('S1',), ('Yp',)): 863, (('S1',), ('Ym',)): 835, (('S2',), ('Zp',)): 1646, (('S2',), ('Xp',)): 870, (('S2',), ('Xm',)): 819, (('S2',), ('Ym',)): 813, (('S2',), ('Yp',)): 802, (('S3',), ('Zp',)): 1744, (('S3',), ('Xm',)): 834, (('S3',), ('Xp',)): 812, (('S3',), ('Yp',)): 819, (('S3',), ('Ym',)): 899, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zp',)): 5029, (('S0',), ('Xm',)): 2495, (('S0',), ('Xp',)): 2561, (('S0',), ('Ym',)): 2582, (('S0',), ('Yp',)): 2480, (('S1',), ('Zp',)): 1636, (('S1',), ('Xm',)): 867, (('S1',), ('Xp',)): 805, (('S1',), ('Yp',)): 866, (('S1',), ('Ym',)): 817, (('S2',), ('Zp',)): 1652, (('S2',), ('Xp',)): 839, (('S2',), ('Xm',)): 822, (('S2',), ('Ym',)): 810, (('S2',), ('Yp',)): 859, ((

{'00': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xp',)): 4999, (('S0',), ('Xm',)): 5001, (('S0',), ('Yp',)): 4937, (('S0',), ('Ym',)): 5063, (('S1',), ('Zp',)): 3356, (('S1',), ('Xp',)): 1683, (('S1',), ('Xm',)): 1727, (('S1',), ('Ym',)): 1706, (('S1',), ('Yp',)): 1584, (('S2',), ('Zp',)): 3320, (('S2',), ('Xm',)): 1686, (('S2',), ('Xp',)): 1598, (('S2',), ('Ym',)): 1685, (('S2',), ('Yp',)): 1674, (('S3',), ('Zp',)): 3385, (('S3',), ('Xp',)): 1621, (('S3',), ('Xm',)): 1679, (('S3',), ('Ym',)): 1671, (('S3',), ('Yp',)): 1651, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '11': {(('S1',), ('Zp',)): 6644, (('S1',), ('Xp',)): 3288, (('S1',), ('Xm',)): 3302, (('S1',), ('Ym',)): 3326, (('S1',), ('Yp',)): 3384, (('S2',), ('Zp',)): 6680, (('S2',), ('Xm',)): 3290, (('S2',), ('Xp',)): 3426, (('S2',), ('Ym',)): 3353, (('S2',), ('Yp',)): 3288, (('S3',), ('Zp',)): 6615, (('S3',), ('Xm',)): 3361, (('S3',), ('Xp',)): 3339, (('S3',), ('Ym',)): 3360, (('S3',

In [95]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [96]:
direct_recombined_dist

{'00000': 0.0005094261613678345,
 '11000': -0.00011284168157260865,
 '00010': 0.032611105681352885,
 '11010': 0.0003049911319855527,
 '00001': 0.02174901563843719,
 '11001': -0.0001221689278517466,
 '00011': 0.9370969743191874,
 '11011': -0.004285963506885474,
 '00110': 0.0002388661602160958,
 '11110': -6.540903198279277e-05,
 '00101': 0.000389390739168302,
 '11101': 9.007645498759152e-05,
 '00111': 0.011139525353440698,
 '11111': 0.00045701150814916923}

In [97]:
likely_recombined_dist

{'00000': 0.0003413764877824195,
 '11000': 2.1597845610063705e-05,
 '00010': 0.03187673136492358,
 '11010': 0.001431488928811807,
 '00001': 0.02146774746356125,
 '11001': 0.0006550578834079355,
 '00011': 0.9182964839969032,
 '11011': 0.014188613736666897,
 '00110': 0.00010284473071243574,
 '11110': 7.4863351441369316e-06,
 '00101': 0.0001945878678214684,
 '11101': 8.884508671674064e-06,
 '00111': 0.010712553075191835,
 '11111': 0.0006945457747911912}

In [98]:
filtered_recombined_lists

[{'0000': 0.07901401476706026,
  '1000': 0.08051473997001983,
  '0010': 0.07784466447046834,
  '1010': 0.07925496563872032,
  '0001': 0.04437429871162705,
  '1001': 0.04525105284867815,
  '0011': 0.04883669995860413,
  '1011': 0.049779219381175584,
  '0100': 0.06965743961360969,
  '1100': 0.07116952813150668,
  '0110': 0.07358411413425844,
  '1110': 0.07504970956421418,
  '0101': 0.04503645290491789,
  '1101': 0.04588410213134209,
  '0111': 0.045615461565815675,
  '1111': 0.04648910482352534},
 {'0000': 0.0026707449592855478,
  '1000': 0.002654232652461254,
  '0010': 0.12752713640945265,
  '1010': 0.12592758989018363,
  '0001': 0.0024455005437636424,
  '1001': 0.0024076984832164835,
  '0011': 0.11021894665108004,
  '1011': 0.10883670908155169,
  '0100': 0.002786318827886236,
  '1100': 0.0027606173152930993,
  '0110': 0.133003412997044,
  '1110': 0.13164927347883895,
  '0101': 0.0025745479400805945,
  '1101': 0.0025606147878520186,
  '0111': 0.11116191520950004,
  '1111': 0.109815306179

In [99]:
from utils.utils import filter_results

In [100]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [101]:
filter_direct_recombined

{'0000': 0.0005094261613678345,
 '0010': 0.032611105681352885,
 '0001': 0.02174901563843719,
 '0011': 0.9370969743191874,
 '0110': 0.0002388661602160958,
 '0101': 0.000389390739168302,
 '0111': 0.011139525353440698}

In [102]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [103]:
filter_likely_recombined

{'0000': 0.0003413764877824195,
 '0010': 0.03187673136492358,
 '0001': 0.02146774746356125,
 '0011': 0.9182964839969032,
 '0110': 0.00010284473071243574,
 '0101': 0.0001945878678214684,
 '0111': 0.010712553075191835}

In [104]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [105]:
norm_filter_dist

{'0000': 0.00034728296356430883,
 '0010': 0.032428260683875144,
 '0001': 0.021839181159270422,
 '0011': 0.9341847953992364,
 '0110': 0.00010462414415474376,
 '0101': 0.0001979546155907803,
 '0111': 0.01089790103430832}

In [106]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [107]:
final_expect_val

(-1.0574188580910053+0j)

In [108]:
import json
with open('LiH_auckland_check3.json', 'w') as f:
    json.dump(norm_filtered_lists, f)